# Đề bài: Sử dụng mạng học sâu để phân loại hình ảnh đất đai từ ảnh vệ tinh
## Model: Unet
## Data: EuroSat-RGB + ESA WorldCover

In [1]:
!rm -rf /kaggle/working/

rm: cannot remove '/kaggle/working/': Device or resource busy


# 1. Clone repo

In [2]:
!git clone https://github.com/yuh-tech/ADCV_2025 /kaggle/working//Final_exam

Cloning into '/kaggle/working//Final_exam'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 80 (delta 32), reused 68 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (80/80), 65.48 KiB | 1.17 MiB/s, done.
Resolving deltas: 100% (32/32), done.


# 2. Cài đặt dependences

In [3]:
!pip install -r /kaggle/working/Final_exam/requirements.txt

INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/

In [4]:
# Check xem file config.py trên Kaggle có code mới chưa
!cat /kaggle/working/Final_exam/config.py | grep -A 3 "REFERENCE_MAPS_FOLDER = DATA_DIR"

    REFERENCE_MAPS_FOLDER = DATA_DIR / "Reference_Maps"
    
    print(f"  📍 Reference Maps: {REFERENCE_MAPS_FOLDER}")
    print(f"     Exists: {REFERENCE_MAPS_FOLDER.exists()}")
--
    REFERENCE_MAPS_FOLDER = DATA_DIR / "Reference_Maps"
    
    # BigEarthNet folders
    BIGEARTHNET_FOLDERS = [


In [5]:
!python /kaggle/working/Final_exam/setup_kaggle.py


Setting up Kaggle Environment for Land Cover Segmentation
✓ Running on Kaggle
⚙ Installing zstd...
✓ zstd installed successfully

Checking Kaggle Datasets
  ✓ BigEarthNet S2 (parts 0-5): /kaggle/input/bigearthnetv2-s2-0
  ✓ BigEarthNet Metadata: /kaggle/input/bigearthnet-s2-metadata
  ✓ BigEarthNet Reference Maps: /kaggle/input/bigearthnet-s2-referencesmap
  ✓ EuroSAT RGB: /kaggle/input/rgbeurosat

✓ All datasets are available!

Extracting reference maps...

Extracting Reference Maps on Kaggle
Extracting to /kaggle/working/data...
This may take several minutes...
✓ Successfully extracted to: /kaggle/working/data/Reference_Maps

Checking Python packages...
  ✓ torch
  ✓ torchvision
  ✓ numpy
  ✓ pandas
  ✓ rasterio
  ✓ albumentations
  ✓ matplotlib
  ✓ seaborn
  ✓ sklearn
  ✓ tqdm

Setup Complete!

You can now run training:
  Stage 1 (EuroSAT): !python train_stage1.py --epochs 50
  Stage 2 (U-Net):   !python train_stage2.py --epochs 50

Outputs will be saved to /kaggle/working/outputs/

# 2. test dữ liệu 


In [6]:
# ============================================================================
# SETUP AND IMPORTS
# ============================================================================

import sys
import os
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import torch
import pandas as pd

# Auto-detect environment
IS_KAGGLE = os.path.exists('/kaggle/input')

if IS_KAGGLE:
    print("Running on Kaggle environment")
    # On Kaggle, repo is cloned to /kaggle/working/Final_exam
    project_root = Path('/kaggle/working/Final_exam')
else:
    print("Running on local environment")
    # On local, notebook is in notebooks/, so go up one level
    project_root = Path.cwd().parent

# Add project root to path
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print(f"Python path updated")


Running on Kaggle environment
Project root: /kaggle/working/Final_exam
Python path updated


In [7]:
# Import configurations and modules
from config import (
    EUROSAT_PATH, METADATA_PATH, BIGEARTHNET_FOLDERS, 
    REFERENCE_MAPS_FOLDER, CORINE_TO_EUROSAT, CLASS_NAMES, NUM_CLASSES,
    IS_KAGGLE
)
from src.data import (
    EuroSATDataset, BigEarthNetSegmentationDataset,
    get_classification_train_augmentation, get_val_augmentation,
    get_segmentation_train_augmentation
)
from src.utils.visualization import mask_to_rgb, denormalize_image, COLOR_PALETTE

print("Imports successful!")
print(f"\n Paths configuration:")
print(f"  EuroSAT: {EUROSAT_PATH}")
print(f"  Exists: {EUROSAT_PATH.exists()}")
print(f"\n  Metadata: {METADATA_PATH}")
print(f"  Exists: {METADATA_PATH.exists()}")
print(f"\n  Reference Maps: {REFERENCE_MAPS_FOLDER}")
print(f"  Exists: {REFERENCE_MAPS_FOLDER.exists()}")
print(f"\n  BigEarthNet folders: {len(BIGEARTHNET_FOLDERS)} found")
for folder in BIGEARTHNET_FOLDERS:
    print(f"    - {folder} (exists: {folder.exists()})")


🌐 Running on Kaggle environment
  📍 Reference Maps: /kaggle/working/data/Reference_Maps
     Exists: True
  📁 Found 6 BigEarthNet folders
  📍 Metadata: True
  📍 EuroSAT: True
Imports successful!

 Paths configuration:
  EuroSAT: /kaggle/input/rgbeurosat/RBG
  Exists: True

  Metadata: /kaggle/input/bigearthnet-s2-metadata/metadata.parquet
  Exists: True

  Reference Maps: /kaggle/working/data/Reference_Maps
  Exists: True

  BigEarthNet folders: 6 found
    - /kaggle/input/bigearthnetv2-s2-0/BigEarthNet-S2 (exists: True)
    - /kaggle/input/bigearthnetv2-s2-1/BigEarthNet-S2 (exists: True)
    - /kaggle/input/bigearthnetv2-s2-2/BigEarthNet-S2 (exists: True)
    - /kaggle/input/bigearthnetv2-s2-3/BigEarthNet-S2 (exists: True)
    - /kaggle/input/bigearthnetv2-s2-4/BigEarthNet-S2 (exists: True)
    - /kaggle/input/bigearthnetv2-s2-5/BigEarthNet-S2 (exists: True)


## 2.1 Test EuroSat dataset

In [8]:
# DEBUG: Check denormalization works correctly
print("="*70)
print("DEBUG: Testing denormalization")
print("="*70)

sample = train_dataset[0]
img = sample['image']
label = sample['label']

print(f"\nSample 0: {CLASS_NAMES[label]}")
print(f"  Before denorm: shape={img.shape}, range=[{img.min():.3f}, {img.max():.3f}]")

img_denorm = denormalize_image(img)
print(f"  After denorm: shape={img_denorm.shape}, type={type(img_denorm)}")
print(f"  After denorm: range=[{img_denorm.min():.3f}, {img_denorm.max():.3f}]")
print(f"  After denorm: mean={img_denorm.mean():.3f}")

# Check if it's displayable
if img_denorm.min() >= 0 and img_denorm.max() <= 1:
    print("   Image is in valid range [0, 1] for display")
else:
    print(f"   Image range issue: [{img_denorm.min()}, {img_denorm.max()}]")


DEBUG: Testing denormalization


NameError: name 'train_dataset' is not defined

In [ ]:
print("="*70)
print("TESTING EUROSAT DATASET")
print("="*70)

# Create transforms
train_transform = get_classification_train_augmentation(64, strength='light')
val_transform = get_val_augmentation(64)

# Load datasets
try:
    train_dataset = EuroSATDataset(EUROSAT_PATH, 'train', train_transform)
    val_dataset = EuroSATDataset(EUROSAT_PATH, 'val', val_transform)
    test_dataset = EuroSATDataset(EUROSAT_PATH, 'test', val_transform)
    
    print(f"✓ Train samples: {len(train_dataset):,}")
    print(f"✓ Val samples: {len(val_dataset):,}")
    print(f"✓ Test samples: {len(test_dataset):,}")
    print(f"✓ Classes: {train_dataset.classes}")
    
    # Get class distribution
    print("\n Class distribution:")
    train_dataset.get_class_distribution()
    
except Exception as e:
    print(f" Error loading EuroSAT: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Alternative visualization - without normalization
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('EuroSAT Training Samples (Raw)', fontsize=16, fontweight='bold')

for i in range(10):
    # Get sample WITHOUT transform
    img_path, label = train_dataset.dataset.samples[i]
    from PIL import Image
    img_pil = Image.open(img_path)
    
    class_name = CLASS_NAMES[label]
    
    ax = axes[i // 5, i % 5]
    ax.imshow(img_pil)
    ax.set_title(f'{class_name}', fontsize=10, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.show()

print("✓ Visualized 10 raw images")

In [ ]:
# Visualize EuroSAT samples
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('EuroSAT Training Samples', fontsize=16, fontweight='bold')

for i in range(10):
    sample = train_dataset[i]
    img = sample['image']  # torch.Tensor (3, 64, 64)
    label = sample['label']
    class_name = CLASS_NAMES[label]
    
    # Denormalize - returns numpy array (H, W, 3)
    img_np = denormalize_image(img)
    
    ax = axes[i // 5, i % 5]
    ax.imshow(img_np)
    ax.set_title(f'{class_name}', fontsize=10, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.show()

print(" Visualized 10 EuroSAT samples")

In [ ]:
# DEBUG: Check Reference Maps structure
print("="*70)
print("DEBUG: Checking Reference Maps Structure")
print("="*70)

import os

ref_maps_root = REFERENCE_MAPS_FOLDER
print(f"\nREFERENCE_MAPS_FOLDER: {ref_maps_root}")
print(f"Exists: {ref_maps_root.exists()}")

if ref_maps_root.exists():
    # List first level
    print(f"\nContents of {ref_maps_root}:")
    items = list(ref_maps_root.iterdir())[:10]
    for item in items:
        print(f"  - {item.name} ({'dir' if item.is_dir() else 'file'})")
    
    # Find tile folders (start with S2A or S2B)
    tile_folders = [d for d in ref_maps_root.iterdir() if d.is_dir() and d.name.startswith('S2')]
    
    if tile_folders:
        print(f"\n✓ Found {len(tile_folders)} tile folders")
        sample_tile = tile_folders[0]
        print(f"\nExample tile folder: {sample_tile.name}")
        
        # Check contents
        ref_maps = list(sample_tile.glob("*_reference_map.tif"))
        print(f"  Contains {len(ref_maps)} reference map files")
        if ref_maps:
            print(f"  Example: {ref_maps[0].name}")
            print(f"\n✅ Reference Maps structure looks correct!")
    else:
        print(f"\n No tile folders found (should start with 'S2A' or 'S2B')")
else:
    print("\n REFERENCE_MAPS_FOLDER does not exist!")
    print("\n Make sure to run setup_kaggle.py first to extract reference maps!")


## 2.2 Test BigEarthNet dataset

In [ ]:
print("="*70)
print("LOADING BIGEARTHNET METADATA")
print("="*70)

try:
    metadata_df = pd.read_parquet(METADATA_PATH)
    print(f" Loaded metadata: {len(metadata_df):,} patches")
    print(f"  Columns: {list(metadata_df.columns)}")
    
    # Show split distribution
    print(f"\n Split distribution:")
    print(metadata_df['split'].value_counts())
    
    # For testing, use a subset
    TEST_SUBSET_SIZE = 100  # Adjust based on your needs
    
    train_df = metadata_df[metadata_df['split'] == 'train'].head(TEST_SUBSET_SIZE)
    val_df = metadata_df[metadata_df['split'] == 'validation'].head(50)
    
    print(f"\n Using {len(train_df)} train samples (subset for testing)")
    print(f" Using {len(val_df)} val samples (subset for testing)")
    
    # Show sample metadata
    print(f"\n Sample metadata:")
    print(train_df.head(3))
    
except Exception as e:
    print(f" Error loading metadata: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
print("="*70)
print("CREATING BIGEARTHNET DATASET")
print("="*70)

# Create transforms
seg_train_transform = get_segmentation_train_augmentation(120, strength='light')
seg_val_transform = get_val_augmentation(120)

try:
    train_dataset_ben = BigEarthNetSegmentationDataset(
        metadata_df=train_df,
        data_folders=BIGEARTHNET_FOLDERS,
        reference_maps_folder=REFERENCE_MAPS_FOLDER,
        corine_to_eurosat_mapping=CORINE_TO_EUROSAT,
        transform=seg_train_transform,
        num_classes=NUM_CLASSES,
        validate_data=True
    )
    
    val_dataset_ben = BigEarthNetSegmentationDataset(
        metadata_df=val_df,
        data_folders=BIGEARTHNET_FOLDERS,
        reference_maps_folder=REFERENCE_MAPS_FOLDER,
        corine_to_eurosat_mapping=CORINE_TO_EUROSAT,
        transform=seg_val_transform,
        num_classes=NUM_CLASSES,
        validate_data=True
    )
    
    print(f"✓ Created BigEarthNet train dataset: {len(train_dataset_ben)} samples")
    print(f"✓ Created BigEarthNet val dataset: {len(val_dataset_ben)} samples")
    
except Exception as e:
    print(f"✗ Error creating BigEarthNet dataset: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
print("="*70)
print("TESTING SAMPLE LOADING")
print("="*70)

# Test loading a few samples
num_test = 5
success = 0

for i in range(num_test):
    try:
        sample = train_dataset_ben[i]
        print(f"\n✓ Sample {i}: {sample['patch_id']}")
        print(f"  Image: {sample['image'].shape}, Mask: {sample['mask'].shape}")
        print(f"  Classes: {torch.unique(sample['mask']).tolist()}")
        success += 1
    except Exception as e:
        print(f"\n✗ Sample {i} failed: {e}")

print(f"\n📊 Loaded {success}/{num_test} samples successfully")


## 2.3 Test DataLoader

In [ ]:
from torch.utils.data import DataLoader

print("="*70)
print("TESTING DATALOADER")
print("="*70)

# Create DataLoader
train_loader = DataLoader(
    train_dataset_ben,
    batch_size=4,
    shuffle=True,
    num_workers=0,  # Use 0 for debugging
    pin_memory=False
)

# Get one batch
try:
    batch = next(iter(train_loader))
    
    print(f"✓ Batch loaded successfully!")
    print(f"  Images: {batch['image'].shape}")
    print(f"  Masks: {batch['mask'].shape}")
    print(f"  Patch IDs: {batch['patch_id']}")
    print(f"\n📊 Data info:")
    print(f"  Image dtype: {batch['image'].dtype}")
    print(f"  Mask dtype: {batch['mask'].dtype}")
    print(f"  Image range: [{batch['image'].min():.3f}, {batch['image'].max():.3f}]")
    print(f"  Mask range: [{batch['mask'].min()}, {batch['mask'].max()}]")
    print(f"  Unique classes: {torch.unique(batch['mask']).tolist()}")
    
except Exception as e:
    print(f"✗ Error: {e}")
    import traceback
    traceback.print_exc()
